### Data

Data used is from the movielens website.

In [1]:
#imports
import numpy as np
from csv import DictReader
import csv
import pandas as pd
import collections


### Reading data:

In [ ]:
#what tables we need from data
# - movie genres
# - avg rating for each movie
# - what movies user has rated and the ratings for each user
# - how much ratings does movie have
# - how much movies has each user rated
# - 

In [4]:
movie_titles = []
mt = {}
movie_genres = {}
genres = set()
movie_references = {}

reader = DictReader(open('data/ml-latest-small/movies.csv', 'rt', encoding='utf-8'))
for row in reader:
    current_row_genres = row["genres"].split("|")
    movie_references[int(row["movieId"])] = []
    for genre in current_row_genres:
        if genre != "(no genres listed)":
            movie_references[int(row["movieId"])].append(str(genre))
    movie_genres[int(row["movieId"])] = current_row_genres
    movie_references[int(row["movieId"])].append(row["title"])
    movie_titles.append((int(row["movieId"]), row["title"]))
    mt[int(row["movieId"])] = row["title"]
    for genr in current_row_genres:
        genres.add(genr)

movie_titles_arr = np.array(movie_titles)

reader = DictReader(open('data/ml-latest-small/tags.csv', 'rt', encoding='utf-8'))
for row in reader:
    movie_references[int(row["movieId"])].append(row["tag"])


df_ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
movies = df_ratings['movieId'].to_numpy().tolist()
movief = dict(collections.Counter(movies))
#df_Rc = pd.DataFrame.from_dict(movief, orient = 'index', columns=['movieId', 'ratingCount'])
dtf = [('movieId', 'int'), ('contributions', 'int')]
movief_arr = np.array(list(movief.items()), dtype = dtf)
df_movies = pd.DataFrame(movief_arr, columns = ['movieId', 'contributions'])
movie_arr = df_movies.to_numpy()
df_Rc = pd.DataFrame(movie_arr, columns=['movieId', 'ratingCount'])
nonRated = []
for movie in movie_titles_arr:
    if int(movie[0]) not in movie_arr[:,0]:
        nonRated.append(int(movie[0]))

df_T = pd.DataFrame(movie_titles_arr, columns = ['movieId', 'title'])
df_Rc['movieId'] = df_Rc['movieId'].astype(int)
df_T['movieId'] = df_T['movieId'].astype(int)
df_TRc = pd.merge(df_Rc, df_T)
print(len(df_TRc), "movies")

users = df_ratings['userId'].to_numpy().tolist()
userf = dict(collections.Counter(users))
dtf = [('userId', 'int'), ('contributions', 'int')]
userf_arr = np.array(list(userf.items()), dtype = dtf)
df_users = pd.DataFrame(userf_arr, columns = ['userId', 'contributions'])
users_arr = df_users.to_numpy()

count = 0
for uporabnik in userf_arr:
    count += uporabnik[1]
print(count, "ratings out of", 9066*671, "slots")
print(genres)
features = len(genres)-1
print(features, "features")
display(df_users)
df_TRc = df_TRc.sort_values(by=['ratingCount'], ascending=False)
display(df_TRc)

9066 movies
100004 ratings out of 6083286 slots
{'Crime', 'Children', 'War', 'Musical', 'Horror', 'Animation', 'Adventure', 'Film-Noir', 'Mystery', 'Romance', 'Sci-Fi', 'IMAX', 'Documentary', 'Fantasy', 'Western', 'Drama', 'Comedy', 'Thriller', 'Action', '(no genres listed)'}
19 features


,userId,contributions
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100
...,...,...
666,667,68
667,668,20
668,669,37
669,670,31


,movieId,ratingCount,title
57,356,341,Forrest Gump (1994)
49,296,324,Pulp Fiction (1994)
99,318,311,"Shawshank Redemption, The (1994)"
92,593,304,"Silence of the Lambs, The (1991)"
143,260,291,Star Wars: Episode IV - A New Hope (1977)
...,...,...,...
6544,98604,1,From Up on Poppy Hill (Kokuriko-zaka kara) (2011)
6545,103659,1,Justice League: The Flashpoint Paradox (2013)
6546,104419,1,Justice League: Crisis on Two Earths (2010)
6547,115927,1,Doctor Strange (2007)


In [6]:
movies_arr = df_Rc.to_numpy()
print(users_arr[:,0])
print(len(users_arr))

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 245 246 24

V spodnjem oknu se sestavi glavna matrika vseh filmov X uporabnikov.
V to matriko se potem dodajajo na novo prijavljeni uporabniki.

In [6]:
df_ratings['movieId'] = df_ratings['movieId'].astype(int)
df_RcRatings = df_ratings
print(len(df_ratings))
RcRatings_arr = df_RcRatings.to_numpy()
movie_users = {}
setOfUsers = set(list(users_arr[:,0]))
for i in range(len(RcRatings_arr)):
    row = RcRatings_arr[i].tolist()
    uId = int(row[0])
    mId = int(row[1])
    if (mId not in movie_users):
            movie_users[mId] = []
    movie_users[mId].append((uId, row[2]))

movie_user = {}
for i in sorted(movie_users.keys()):
    movie_user[i] = movie_users[i]

howManyRatings = 0
for film in movie_user:
    howManyRatings += len(movie_user[film])
print(howManyRatings, " ratings")

Xlist = []
flag1 = True
matches = 0
matrix_users = {}
matrix_usersr = {}
matrix_movies = {}
matrix_moviesr = {}
im = 0
iu = 0
for movie in movie_user:
    row = []
    matrix_moviesr[movie] = im
    matrix_movies[im] = movie
    im += 1
    if im % 100 == 0:
        print('\rGenerating matrix %d/%d' % (im, 9066), end="")
    for user in users_arr[:, 0]:
        if flag1:
            matrix_users[iu] = user
            matrix_usersr[user] = iu
            iu += 1
        flg = True
        for usr, rting in movie_user[movie]:
            if int(usr) == int(user):
                row.append(rting)
                matches += 1
                flg = False
                break
        if flg:
            row.append(0)
    flag1 = False
    #Xlist.append(row)
    Xlist.append(np.array(row))
matriX = np.array(Xlist)
##cnt = 0
print("                               ")
if matches == 100004:
    print("Matrix generated")
print(matriX)

100004
100004  ratings
Generating matrix 9000/9066                               
Matrix generated
[[0. 0. 0. ... 0. 4. 5.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


V tem oknu je koda za uporabniški vmesnik.
Na začetku se pojavi napis:  

"-----------------------------------------------------"  
         "Welcome to the MovieRecommender!"  
"-----------------------------------------------------"  
"press 1. Login"  
"press 2. Register"  
  
Tukaj uporabnik izbere prijavo ali registracijo s tem da v lupino vtipka 1 ali 2.  
Za testiranje so vsa uporabniska imena in gesla na ze obstojecih uporabnikih nastavljena na "NA"  
Nov uporabnik pa si lahko ustvari svoje uporabnisko ime in geslo.  
Za prijavo mora uporabiti svoj id, namesto uporabniskega imena  
  
  
Po vspešni prijave se prikaže meni:  
"options:"  
"-----------------------------------------------------"  
"press 1 to get recommendations"  
"press 2 to get your watched movies"  
"press 3 to recommend by genre"  
"press 4 to recommend by input"  
"press l to log out"  
"press d to delete account"  
"press q to quit"  
"-----------------------------------------------------"  
"option:"  

Tukaj uporabnik vnese opcijo (1, 2, 3, 4, l, d, q) oz. (rekomendacije sitema, ze ogledani filmi, rekomendacija po žanru,  
rekomendacija gelde na vnos uporabnika, odjava, izbris uporabnika, izhod iz aplikacije)  

Pri vseh opcijah se naredi matrika, ki napove ocene uporabnikov, s pomočjo matične faktorizacije in gradialnega spusta  
več pri naslednjem oknu...  

In [1]:
#user interface
user_password = {}
for i in range(users_arr.shape[0]):
    user_password[users_arr[i, 0]] = ("NA", "NA")

takenIds = set(list(user_password.keys()))
allIds = set()
for i in range(1, 9999):
    allIds.add(i)
freeIds = allIds - takenIds

mids = {}
for i in range(users_arr.shape[0]):
    user_password[users_arr[i, 0]] = ("NA", "NA")

user_watched_movies = {}
for i in range(users_arr.shape[0]):
    user_watched_movies[users_arr[i, 0]] = []
movieCount= 0
for row in matriX:
    for r in range(len(row)):
        if row[r] != 0:
            user_watched_movies[int(matrix_users[r])].append(matrix_movies[movieCount])
    movieCount += 1

def insertWatchedMovies(mIds_ratings, user):
    for mId, rating in mIds_ratings:
        matriX[matrix_moviesr[mId], matrix_usersr[user]]= rating

def startScreen():
    print("Welcome to movie recommender")
    logIn()

def logOut():
    startScreen()

def logIn():
    print("Log in: enter userId and password")
    userId = input("UserId: ")
    password = input("Passwrord: ")
    if userId in user_password:
        if user_password[userId][1] == password:
            print("wellcome back", user_password[userId][0], "your id is", userId)
            print("#######################################################")
            print("options:")
            print ("input number of recommendations you want:")
            numOfRec = input()
            print("press 1 to get recommendations")
            print("press 2 to get your watched movies")
            print("press 3 to recommend by genre")
            print("press 4 to recommend by input")
            print("press l to log out")
            print("press d to delete account")
            print("#######################################################")
            option = input("option:")
            if option == "1":
                print("insert acuracy 1-10 (number of features for matrix factorization) higher takes longer")
                accuracy = input()
                get_recomendations(userId, numOfRec, "", "", accuracy, 1)
            if option == "2":
                print(user_watched_movies[userId])
            if option == "3":
                print("insert acuracy 1-10 (number of features for matrix factorization) higher takes longer")
                accuracy = input()
                print("enter genre")
                genre = input()
                get_recomendations(userId, numOfRec, genre, "", accuracy, 3)
            if option == "4":
                print("insert acuracy 1-10 (number of features for matrix factorization) higher takes longer")
                accuracy = input()
                print("enter your reference")
                reference = input()
                get_recomendations(userId, numOfRec, "", reference, accuracy, 4)
            if option == "l":
                logOut()
            if option == "d":
                deleteAccount(userId)

        else:
            print("wrong password")
    else:
        print("user not found, would you like to register? (y/n)")
        choice = input()
        if choice == "y":
            print("Sign in: enter user name and password")
            userName = input("User name: ")
            password = input("Passwrord: ")
            addUser(userName, password)
        else:
            print("goodbye")

def addUser(userName, password):
    newId = freeIds.pop()
    user_password[newId] = (userName, password)
    users_arr.tolist().append((newId, 0))
    for row in matriX:
        row.tolist().append(0)
    print("account created succesfuly")
    print("your id is", newId)
    print("would you like to log in? (y/n)")
    answer = input()
    if answer == "y":
            logIn()
    else:
        print("see you later")

def deleteAccount(userId):
    print("are you sure you want to delete", user_password[userId][0], "? (y/n)")
    answer = input()
    if answer == "y":
        for row in matriX:
            row[matrix_usersr[userId]] = 0
        user_password[userId] = ("NA", "NA")
        freeIds.add(userId)
        print("account deleted")
    else:
        print("deletion aborted")

NameError: name 'users_arr' is not defined

V spodnjem oknu (def getSample) se sestavi naključna matrika poljubne velikosti m * n, ki je strogo manjsa od zgornje,  
ker drugače predolgo traja, da se iskanje izteče.  
Zagotevljeno je, da se uporabnik, za katerega se matrika sestavlja nahaja v vzorcu.  


In [7]:
xdict = {}
for i in range(len(matriX)):
    xdict[matrix_movies[i]] = matriX[i].tolist()


def getSample(m, n, rcM, rcU, userId):
    indexUsers = {}
    indexMovies = {}
    unwatched = {}
    ##movie sample                               m
    ##user sample                                n
    ##min amount of ratings should movie have    rcM
    ##min amount that user rated                 rcU
    Xsample = []
    df_users_over_rcU = df_users[df_users['contributions'] >= rcU]
    df_movies_over_rcM = df_TRc[df_TRc['ratingCount'] >= rcM]

    temp = np.array(df_users_over_rcU['userId']).tolist()
    for i in range(0, len(temp)-1):
        if temp[i] == userId:
            del temp[i]
            break
    tempnp = np.array(temp)
    df_users_over_rcU = pd.DataFrame(tempnp, columns=['userId'])
    df_users_sample = df_users_over_rcU.sample(n=n-1, replace=False)

    df_movies_sample = df_movies_over_rcM.sample(n=m, replace=False)
    movies_sample = df_movies_sample['movieId'].to_numpy()
    users_sample = df_users_sample['userId'].to_numpy()
    users_sample = np.append(users_sample, userId)
    for mo in range(len(movies_sample)):
        row = []
        indexMovies[movies_sample[mo]] = mo
        cnt = 0
        for i in range(len(xdict[movies_sample[mo]])):
            if matrix_users[i] in users_sample:
                cnt += 1
                rating = xdict[movies_sample[mo]][i]
                indexUsers[matrix_users[i]] = cnt
                if rating == 0 and matrix_users[i] == userId:
                    unwatched[mo] = movies_sample[mo]
                row.append(rating)
        Xsample.append(np.array(row))
    return np.array(Xsample), indexMovies, indexUsers, unwatched
m = 50
n = 50
rcM = 0
rcU = 0
a, b, c, d = getSample(m, n, rcM, rcU, 8)
print(a)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


V spodnjem oknu pa se zgodi Matrična faktorizacija.  
Generirata se 2 matriki H in W, H kot število uporabnikov v vzorcu * k  
                                W kot število filmov * k  
                                
K si uporabnik izbere in predstavlja število featurjev oz. parametrov.  
Poskusil se z  večimi k-ji, okoli 10-13, se mi je zdelo, da se nekako vrednosti najbolj ujeemajo z vzorcem.  
Matriki se nato zmnožita in sestavita matriko, ki je lahko blizu, ali ne tako blizu našega vzorca.  
Izračuna se napaka (razlika med izračunano matriko in vzorčno matriko)
parameter lr predstavlja learning rate, rp, pa parameter za reguliranje  
Izračun matrik se izvede 1000*.

In [76]:
def getHandW(X, H, W, K, steps=1000, lr=0.0002, rp=0.02):
    stepsForPercent = steps / 100
    count = 0
    for step in range(steps):
        if step % stepsForPercent == 0:
            count += 1
            print("\rSearching... ", count, "%", end="")
        for i in range(len(X)):
            for j in range(len(X[i])):
                if X[i, j] > 0:
                    error = X[i, j] - np.dot(H[i,:],W[:,j])

                    for k in range(K):
                        H[i, k] = H[i, k] + lr * (2 * error * W[k, j] - rp * H[i, k])
                        W[k, j] = W[k, j] + lr * (2 * error * H[i, k] - rp * W[k, j])
        e = 0
        for i in range(len(X)):
            for j in range(len(X[i])):
                if X[i, j] > 0:
                    e = e + pow(X[i, j] - np.dot(H[i,:],W[:,j]), 2)
                    for k in range(K):
                        e = e + (rp/2) * (pow(H[i, k],2) + pow(W[k, j],2))
        if e < 0.001:
            break
    return H, W

def recommendToUser(userId, sampleMovies, sampleUsers, rcM, rcU, accuracy):
    K = accuracy
    X, movies, users, unwatched =  getSample(sampleMovies, sampleUsers, rcM, rcU, userId)
    X = X.T
    n, m = X.shape
    H = np.random.rand(n,K)
    W = np.random.rand(K,m)

    nH, nW = getHandW(X, H, W, K)
    predictedXt = nH.dot(nW)
    print("")
    print(predictedXt.T)
    print(X.T)


    #############################
    #    movie movie movie movie#
    #user                       #
    #user                       #
    #user                       #
    #user                       #
    #user                       #
    #############################

    unwatched2 = {}
    for i in range(len(predictedXt[users[userId]])):
        if i in unwatched:
            unwatched2[predictedXt[users[userId]][i]] = unwatched[i]
    return unwatched2


def get_recomendations(userId, numOfRec, genre, reference, accuracy, option):
    unsorted = recommendToUser(userId, 100, 100, 100, 100, accuracy)
    sorted_x = []
    if option == 1:
        sorted_x = sorted(unsorted.keys(), reverse=True)


    if option == 3:
        sorted_keys = sorted(unsorted.keys(), reverse=True)
        for i in range(len(sorted_keys)):
            if genre in movie_genres[sorted_keys[i]]:
                sorted_x.append(sorted_keys[i])

    if option == 4:
        movieId_score = {}
        for i in range(len(sorted_keys)):
            movieId_score[sorted_keys[i]] = i
            for ref in movie_references[sorted_keys[i]]:
                if reference in ref:
                    score += 1
        {k: v for k, v in sorted(movieId_score.items(), key=lambda item: item[1])}
        count = 0
        for i in movieId_score:
            count += 1
            if count > numOfRec:
                break
        print(mt[i])

    count = 0
    for i in sorted_x:
        count += 1
        if count > numOfRec:
            break
        print(mt[unsorted[i]])

Funkcija reccomendToUser pokliče funkcijo, ki s pomočjo gradialnega spusta izračuna matriki H, W  
in vrne slovar {ocena : id filma} ta slovar se potem uredi po ocenah.  
1 opcija izpiše le prvih n filmov (n izbere uporabnik).  
3 opcija izpiše prvih n filmov, ki vsebujejo željen žaner.  
4 opcija pa izpišt prvih n filmov, ki imajo najvišjo oceno za ujemanje.  
  
Oceno za ujemanje se izračuna z ujemanji besede z nizi povezani s filmom (Naslov filma, žanri filma, nizi iz tabele tags).  
Algoritem gleda vsak film od prvega do zadnjega, in primerja niz, ki ga je vtipkal uporabnik z referencami (zgornja poved v oklepaju).  
za vsako ujemanje se oceni prišteje 1.  
Algoritem nato izpiše prvih n filmov z najvišjo oceno ujemanja.  